A notebook to quickly iterate and make sure the llama models are loading and working OK

In [ ]:
from tqdm.autonotebook import tqdm
import copy
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForMaskedLM, AutoModelForCausalLM, LlamaTokenizer, LlamaForCausalLM

from transformers import GenerationConfig

## load

In [ ]:
# model_repo = "decapoda-research/llama-7b-hf"
model_repo = "Neko-Institute-of-Science/LLaMA-7B-HF"
model_repo = "elinas/llama-13b-hf-transformers-4.29"

# lora_repo = "tloen/alpaca-lora-7b"
lora_repo = "NousResearch/gpt4-x-vicuna-13b"

# model_repo = "TheBloke/wizardLM-7B-HF"
# lora_repo = None
tokenizer = AutoTokenizer.from_pretrained(model_repo)
model = AutoModelForCausalLM.from_pretrained(model_repo, device_map="auto", 
    load_in_8bit=True,
    torch_dtype=torch.float16)
# if lora_repo is not None:
#     # https://github.com/tloen/alpaca-lora/blob/main/generate.py#L40
#     from peft import PeftModel
#     model = PeftModel.from_pretrained(
#         model, 
#         lora_repo, 
#         torch_dtype=torch.float16,
#         device_map='auto'#{'': 0}
#     )
tokenizer, model

In [ ]:
def format_imdb(text, label):
    return f"""Review: "I think this is a lovely family movie. There are plenty of hilarious scenes and heart-warming moments to be had throughout the movie. The actors are great and the effects well executed throughout. Danny Glover plays George Knox who manages the terrible baseball team 'The Angels' and is great throughout the film. Also fantastic are the young actors Joseph Gordon-Levitt and Milton Davis Jr. Christopher Lloyd is good as Al 'The Angel' and the effects are great in this top notch Disney movie. A touching and heart-warming movie which everyone should enjoy."
Question: Is this review positive? 
Answer: 1
---
Review: " Although Hypnotic isn't without glimmers of inspiration, the ultimate effect of this often clunky crime caper will be to leave you feeling rather sleepy."
Question: Is this review positive?
Answer: 0
---
Review: "A galactic group hug that might squeeze a little too tight on the heartstrings, the final Guardians of the Galaxy is a loving last hurrah for the MCU's most ragtag family."
Question: Is this review negative?
Answer: 0
---
Review: "{text}"
Question: Is this review {'positive' if label else 'negative'}?
Answer: 
"""

In [ ]:
# from https://github.com/deep-diver/LLM-As-Chatbot/blob/main/configs/response_configs/default.yaml
generation_config = GenerationConfig(
    temperature=0.95,
    top_p=0.9,
    top_k=50,
    num_beams=1,
    use_cache=True,
    repetition_penalty=1.2,
    max_new_tokens=512,
    do_sample=True,
)

input_text = format_imdb("The room is the worst movie ever", 0)
# print(input_text)

In [ ]:
# see https://github.com/deep-diver/LLM-As-Chatbot/blob/216abb559d00a0555f41a1426ac9db6c1abc24f3/gens/batch_gen.py#L3
input_ids = tokenizer(input_text, 
                          return_tensors="pt",
#                           truncation=True, 
#                           padding=True,
#                           max_length=600,
                        #   add_special_tokens=False,
                         ).input_ids.to(model.device)

with torch.no_grad():
    generation_output = model.generate(
        input_ids=input_ids, generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
            # max_new_tokens=max_new_tokens,
    )

s = generation_output.sequences[0]
torch.cuda.empty_cache()    
# text_q = tokenizer.batch_decode(input_ids, 
#                                 skip_prompt=True, skip_special_tokens=True
#                                 )
text_ans = tokenizer.decode(s,
                                  #skip_prompt=True, skip_special_tokens=True
                                  )
# print(text_q[0])
print('='*40+'answ'+'='*40)
print(text_ans)

In [ ]:
text_ans = tokenizer.decode(s,
                                #   skip_prompt=True, 
                                #   skip_special_tokens=True
                                  )
print(text_ans)